In [1]:
pip install sentence_transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
from sentence_transformers import SentenceTransformer, models

MODEL = '../../models/word_embeddings/fine_tuned/NER/NCBI-disease'
OUTPUT_MODEL = '../../models/sentence_embeddings/fine_tuned/NLI'
# model_save_path = './models/clinical_bert/finetuned/output2'

DATASET = '../../datasets/NLI/snli_1.0/'


word_embedding_model = models.Transformer(MODEL)

tokens = ["dyspnea", "fagner", "jessica"]
word_embedding_model.tokenizer.add_tokens(tokens, special_tokens=True)
word_embedding_model.auto_model.resize_token_embeddings(len(word_embedding_model.tokenizer))

pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
word_embedding_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
# word_embedding_model.save('/content/drive/MyDrive/Colab Notebooks/models/clinical_bert/output')

Some weights of BertModel were not initialized from the model checkpoint at ../../models/word_embeddings/fine_tuned/NER/NCBI-disease and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
import csv, logging

from sentence_transformers import InputExample


# Convert the dataset to a DataLoader ready for training
logging.info("Read STSbenchmark train dataset")

# sts_dataset_path = '/content/drive/MyDrive/Colab Notebooks/datasets/snli/'


train_samples = []
dev_samples = []
test_samples = []

with open(DATASET + 'snli_1.0_train.txt', "r") as f:
#     labels = f.readlines()
#     print(f.readlines())
    reader = csv.DictReader(f, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
#         print(row)
#         score = float(row['score']) / 5.0  # Normalize score to range 0 ... 1

        gold_label = row['gold_label']  # Normalize score to range 0 ... 1
        float_gold_label = 0
        
        if gold_label == 'contradiction':
            float_gold_label = 0 / 3.0
        if gold_label == 'entailment':
            float_gold_label = 1 / 3.0
        if gold_label == 'neutral':
            float_gold_label = 2 / 3.0
        
        
#         print(gold_label)
        inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=float_gold_label)

#         if row['split'] == 'dev':
        train_samples.append(inp_example)
#         elif row['split'] == 'test':
#             test_samples.append(inp_example)
#         else:
#             train_samples.append(inp_example)

    
print(train_samples[0])
# with gzip.open(sts_dataset_path, 'rt', encoding='utf8') as fIn:
# reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
# print(word_embedding_model.fc1(x).size()) 


<InputExample> label: 0.6666666666666666, texts: A person on a horse jumps over a broken down airplane.; A person is training his horse for a competition.


In [8]:
import math 

from sentence_transformers import losses
from torch.utils.data import DataLoader

from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

num_epochs = 4
train_batch_size = 4

# model_save_path = 'output/training_stsbenchmark_continue_training-'+model_name+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# OUTPUT_MODEL = '../../models/sentence_embeddings/fine_tuned/NLI'
OUTPUT_MODEL = './models/fine_tuned/NLI'

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=word_embedding_model)


# Development set: Measure correlation between cosine score and gold labels
logging.info("Read SNLIbenchmark dev dataset")
# evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))


# Train the model
word_embedding_model.fit(train_objectives=[(train_dataloader, train_loss)],
#           evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=OUTPUT_MODEL)


##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

# model = SentenceTransformer(model_save_path)
# test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
# test_evaluator(model, output_path=model_save_path)


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/137538 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


KeyboardInterrupt: 